In [18]:
import pandas as pd
import numpy as np
import os
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input

In [2]:
train_data = 'images/train'
test_data = 'images/test'

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [5]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(train_data)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(test_data)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(test)

                              image     label
0       images/test\angry\10052.jpg     angry
1       images/test\angry\10065.jpg     angry
2       images/test\angry\10079.jpg     angry
3       images/test\angry\10095.jpg     angry
4       images/test\angry\10121.jpg     angry
...                             ...       ...
7061  images/test\surprise\9806.jpg  surprise
7062  images/test\surprise\9830.jpg  surprise
7063  images/test\surprise\9853.jpg  surprise
7064  images/test\surprise\9878.jpg  surprise
7065   images/test\surprise\993.jpg  surprise

[7066 rows x 2 columns]


In [9]:
from tqdm.notebook import tqdm

In [10]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [11]:
train_features = extract_features(train['image']) 

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\lenovo\anaconda3\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [12]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0
x_test = test_features/255.0

In [14]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [15]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [16]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [19]:
model = Sequential()

# input layers
model.add(Input(shape=(48, 48, 1)))

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [22]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(x=x_train, y=y_train, batch_size=100, epochs=50, validation_data=(x_test,y_test)) 

Epoch 1/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 501s 2s/step - accuracy: 0.2385 - loss: 1.8346 - val_accuracy: 0.2583 - val_loss: 1.8118
Epoch 2/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 511s 2s/step - accuracy: 0.2524 - loss: 1.8069 - val_accuracy: 0.2893 - val_loss: 1.7373
Epoch 3/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 500s 2s/step - accuracy: 0.2931 - loss: 1.7328 - val_accuracy: 0.3691 - val_loss: 1.5996
Epoch 4/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 501s 2s/step - accuracy: 0.3428 - loss: 1.6348 - val_accuracy: 0.4227 - val_loss: 1.4805
Epoch 5/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 494s 2s/step - accuracy: 0.3974 - loss: 1.5372 - val_accuracy: 0.4723 - val_loss: 1.4016
Epoch 6/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 495s 2s/step - accuracy: 0.4238 - loss: 1.4727 - val_accuracy: 0.4871 - val_loss: 1.3417
Epoch 7/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 484s 2s/step - accuracy: 0.4452 - loss: 1.4289 - val_accuracy: 0.5137 - val_loss: 1.2886
Epoch 8/50
289/289 ━━━━━━━━━━━━━━━━━━━━ 494s 2s/step - accuracy: 0.4722 - loss: 1.3754 - val_accu

In [25]:
model_json = model.to_json()
with open('emotiondetector.json', 'w') as json_file:
    json_file.write(model_json)
model.save('emotiondetector.keras')

In [27]:
from keras.models import load_model

model = load_model('emotiondetector.keras')

In [28]:
labels = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [29]:
def ef(image):
    img = load_img(image,grayscale=True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [34]:
image = 'images/train/happy/20000.jpg'
img = ef(image)
pred = model.predict(img)
pred_label = labels[pred.argmax()]
print("Model's Prediction: ",pred_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Model's Prediction:  happy
